In [ ]:
import tensorflow as tf
import keras
import sys
from matplotlib import pyplot
from keras.models import Sequential, load_model, Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras.layers import Dense, Input, Conv2DTranspose
from tensorflow.keras.layers import Flatten, Add, AveragePooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import SGD
from keras.initializers import glorot_uniform
from keras.activations import relu, softmax
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot
from keras import backend as K
from sklearn.metrics import confusion_matrix
from matplotlib import animation, rc
import matplotlib.patches as mpatches
from IPython.display import HTML, display, clear_output, SVG
from tensorflow.keras.optimizers import Adadelta
import ipywidgets as widgets
from tensorflow.keras.applications.resnet50 import ResNet50
import random

In [ ]:
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

In [ ]:
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model


In [ ]:
def model():
	base_model = ResNet50(include_top=False, input_shape=(128, 128, 3), pooling='max')
	
	for layer in base_model.layers:
		layer.trainable = False
	model = Sequential()
	model.add(Conv2DTranspose(3, (3, 3), strides=2, padding='same', activation='relu', input_shape=(32,32,3)))
	model.add(BatchNormalization())
	model.add(Conv2DTranspose(3, (3, 3), strides=2, padding='same', activation='relu'))
	model.add(BatchNormalization())
	model.add(base_model)
	model.add(Flatten())
	model.add(Dense(1024, activation='relu'))
	model.add(Dense(512, activation='relu'))
	model.add(Dense(10, activation='softmax'))
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	
	return model

In [ ]:
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

def preprocess_image_input(input_images):
	input_images = input_images.astype('float32')
	output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
	return output_ims


In [ ]:
x_train, y_train, x_test, y_test = load_dataset()
models = []
# x_train, x_test = prep_pixels(x_train, x_test)
x_train = preprocess_image_input(x_train)
x_test = preprocess_image_input(x_test)
excluded_data_x = []
excluded_data_y = []
def run_test_harness():
  base_epochs = 1
  curr_model = 1 
  for i in range(0, x_train.shape[0], 5000):
      curr_x_train = x_train[i : (i+5000)]
      curr_y_train = y_train[i : (i+5000)]
      model_i = model()
      print("Training shard: " + str(curr_model) + " out of " + str((int)(x_train.shape[0] / 5000)))
      excluded_list = []
      excluded_list.append(random.randint(1,20))
      rand2 = random.randint(1,20)
      while rand2 == excluded_list[0]:
          rand2 = random.randint(1,20)
      excluded_list.append(rand2)

      for j in range(250, len(curr_x_train), 250):
          print("Training slice: " + str((int)(j/250)) + " out of " + str((int)(len(curr_x_train)/250)))
          if ((int)(j/250)) in excluded_list:
              excluded_data_x.append(curr_x_train[j-250 : j])
              excluded_data_y.append(curr_y_train[j-250 : j])
              print("Data excluded for "+ str((int)(j/250)))
              continue
          slice_x = curr_x_train[0: j]
          slice_y = curr_y_train[0: j]
          epochs = base_epochs * (int)(j/250)

          model_i.fit(slice_x, slice_y, batch_size=16, epochs=epochs, verbose=True)
      
      models.append(model_i)
      curr_model+=1

  return models


170508288/170498071 [==============================] - 2s 0us/step


In [ ]:
# models = run_test_harness()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
models = np.load("/content/drive/MyDrive/Classroom/TML outputs /models.npy", allow_pickle=True)

In [ ]:
def predictOnModels(test):
  answers = []
  for i in range(test.shape[0]):
    preds = []
    for j in range(models.shape[0]):
      pred = np.argmax(models[j].predict(test[i].reshape((-1,32,32,3))))
      preds.append(pred)
    ans = np.bincount(preds).argmax()
    answers.append(ans)
  
  return np.array(answers)

def testOnModels(testx, testy):
  # print(testy[0])
  testy = np.argmax(testy, axis=1)
  preds = predictOnModels(testx)
  print(testy.shape)
  return (np.sum(testy ==preds)/preds.shape[0])


In [ ]:
answers = []
for i in range(x_train.shape[0]):
  preds = []
  for j in range(models.shape[0]):
    pred = np.argmax(models[j].predict(x_train[i].reshape((-1,32,32,3))))
    preds.append(pred)
  ans = np.bincount(preds).argmax()
  answers.append(ans)
  break
  

In [ ]:
preds

[6, 6, 4, 7, 6, 7, 6, 5, 5, 6]

In [ ]:
ans

6

In [ ]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [ ]:
testOnModels(x_test, y_test)
# predictOnModels(x_train[0:50])

KeyboardInterrupt: ignored

In [ ]:
# preds = []
# for i in range(models.shape[0]):
#   print(i+1)
#   for j in range(x_test.shape[0]):
#     preds.append(np.argmax(models[i].predict(x_test[j].reshape((-1,32,32,3)))))

preds = []
for i in range(models.shape[0]):
  print(str(i+1))
  pred = models[i].predict(x_test)
  pred = np.argmax(pred, axis=1)
  preds.append(pred)



1
2
3
4
5
6
7
8
9
10


In [ ]:
preds = np.array(preds)

In [ ]:
final_preds = []
for i in range(preds[0].shape[0]):
  inter = []
  for j in range(10):
    inter.append(preds[j][i])
  ans = np.bincount(inter).argmax()
  final_preds.append(ans)


In [ ]:
final_preds = np.array(final_preds)

In [ ]:
x_attack_out = np.vstack((x_train[500:750], x_train[2250:2500]))
x_attack_in = x_train[0:500]

In [ ]:
answer = []
label = []
for i in range(x_attack_out.shape[0]):
  print(i)
  pred = models[0].predict(x_attack_out[i].reshape((-1,32,32,3)))
  for j in range(1,10):
    pred = np.add(pred, models[j].predict(x_attack_out[i].reshape((-1,32,32,3))))
  pred = pred/10
  answer.append(pred)
  label.append(0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
for i in range(x_attack_in.shape[0]):
  pred = models[0].predict(x_attack_in[i].reshape((-1,32,32,3)))
  for j in range(1,10):
    pred = np.add(pred, models[j].predict(x_attack_in[i].reshape((-1,32,32,3))))
  pred = pred/10
  answer.append(pred)
  label.append(1)

(1000, 1, 10)

In [ ]:
best_t = 0.0
best_acc = 0.0
best_adv = 0.0
pv = np.array(answer)
pv = pv.reshape((-1, 10))
in_out_targets = np.array(label)
import numpy as np
import sklearn.metrics as metrics
for i in range(1,100,1):
  threshold =i/100
  in_or_out_pred = np.zeros((pv.shape[0],))
  for i in range(0,len(pv)):
      largest_index = np.argmax(pv[i])        
      if pv[i][largest_index] > threshold:
          in_or_out_pred[i] = 1
      else:
          in_or_out_pred[i] = 0

  cm = metrics.confusion_matrix(in_out_targets, in_or_out_pred)
  accuracy = np.trace(cm) / np.sum(cm.ravel())
  tn, fp, fn, tp = cm.ravel()
  tpr = tp / (tp + fn)
  fpr = fp / (fp + tn)
  advantage = tpr - fpr
  # print(accuracy)
  # print(advantage)
  if advantage > best_adv:
    best_acc = accuracy
    best_adv = advantage

In [ ]:
best_acc

0.526

In [ ]:
best_adv

0.05199999999999999

In [ ]:
best_t

0.0